# Import Packages

In [1]:
import sys
sys.path.append('..')

import utils
import pandas as pd
import numpy as np

from gekko import GEKKO

# Import Data



In [2]:
# Import Forecasts from File System
forecasts = pd.read_csv(utils.get_project_root()+'/03_Forecasting_LightGBM/forecasts/forecasts.csv')
forecasts

,Week,Year,Site_ID,SKU,Prediction,Turnover
0,47,2020,101,1014557,30.868194,29.9
1,47,2020,101,1014824,24.796745,0.0
2,47,2020,101,1016525,56.998503,0.0
3,47,2020,107,1016525,53.501745,0.0
4,47,2020,590,1016525,55.250172,0.0
...,...,...,...,...,...,...
59869,47,2020,102,1281324700,0.767912,0.0
59870,47,2020,106,1281324700,0.767912,0.0
59871,47,2020,115,1281324700,0.767912,0.0
59872,47,2020,117,1281324700,0.767912,0.0


# Optimization (Example based on Article 1016525)

Optimization is based on section "Mixed Integer Nonlinear Programming" in https://gekko.readthedocs.io/en/latest/examples.html

In [3]:
sku = 1016525
supply = 300 # stock at distribution center

In [4]:
forecasts['Prediction'] = forecasts['Prediction']
forecasts.loc[forecasts.SKU == sku]

,Week,Year,Site_ID,SKU,Prediction,Turnover
2,47,2020,101,1016525,56.998503,0.0
3,47,2020,107,1016525,53.501745,0.0
4,47,2020,590,1016525,55.250172,0.0
5429,47,2020,102,1016525,52.571911,0.0
5430,47,2020,103,1016525,34.659034,0.0
5431,47,2020,106,1016525,47.174494,0.0
5432,47,2020,115,1016525,35.900833,0.0
5433,47,2020,116,1016525,45.830675,0.0
26946,47,2020,117,1016525,8.075953,0.0


In [5]:

df = forecasts.loc[forecasts.SKU == sku]

demand = df.Prediction.sum()

if supply >= demand:
    raise ValueError('supply > demand -> no optimization needed.')
    

c_g101 = float(df.loc[df.Site_ID == 101]['Prediction'])
c_g102 = float(df.loc[df.Site_ID == 102]['Prediction'])
c_g103 = float(df.loc[df.Site_ID == 103]['Prediction'])
c_g106 = float(df.loc[df.Site_ID == 106]['Prediction'])
c_g107 = float(df.loc[df.Site_ID == 107]['Prediction'])
c_g115 = float(df.loc[df.Site_ID == 115]['Prediction'])
c_g116 = float(df.loc[df.Site_ID == 116]['Prediction'])
c_g117 = float(df.loc[df.Site_ID == 117]['Prediction'])
c_g590 = float(df.loc[df.Site_ID == 590]['Prediction'])


m = GEKKO(remote=False) 
m.options.SOLVER=1
m.solver_options = ['minlp_maximum_iterations 500', \
                # minlp iterations with integer solution
                'minlp_max_iter_with_int_sol 10', \
                # treat minlp as nlp
                'minlp_as_nlp 0', \
                # nlp sub-problem max iterations
                'nlp_maximum_iterations 50', \
                # 1 = depth first, 2 = breadth first
                'minlp_branch_method 1', \
                # maximum deviation from whole number
                'minlp_integer_tol 0.05', \
                # covergence tolerance
                'minlp_gap_tol 0.00001']

x_g101 = m.Var(lb=1,ub=c_g101, integer=True)
x_g102 = m.Var(lb=1,ub=c_g102, integer=True)
x_g103 = m.Var(lb=1,ub=c_g103, integer=True)
x_g106 = m.Var(lb=1,ub=c_g106, integer=True)
x_g107 = m.Var(lb=1,ub=c_g107, integer=True)
x_g115 = m.Var(lb=1,ub=c_g115, integer=True)
x_g116 = m.Var(lb=1,ub=c_g116, integer=True)
x_g117 = m.Var(lb=1,ub=c_g117, integer=True)
x_g590 = m.Var(lb=1,ub=c_g590, integer=True)

m.Equation(x_g101 + x_g102 + x_g103 + x_g106 + x_g107 + x_g115 + x_g116 + x_g117 + x_g590 == supply)

m.Minimize((c_g101/x_g101) + (c_g102/x_g102) + (c_g103/x_g103)+ (c_g106/x_g106) + (c_g107/x_g107)+ 
      (c_g115/x_g115) + (c_g116/x_g116) + (c_g117/x_g117)+ (c_g590/x_g590))

m.solve(disp=False)


In [7]:
print('x_g101: ' + str(x_g101.value))
print('x_g102: ' + str(x_g102.value))
print('x_g103: ' + str(x_g103.value))
print('x_g106: ' + str(x_g106.value))
print('x_g107: ' + str(x_g107.value))
print('x_g115: ' + str(x_g115.value))
print('x_g116: ' + str(x_g116.value))
print('x_g117: ' + str(x_g117.value))
print('x_g590: ' + str(x_g590.value))

print('Objective: ' + str(m.options.objfcnval))

x_g101: [40.0]
x_g102: [38.0]
x_g103: [31.0]
x_g106: [36.0]
x_g107: [39.0]
x_g115: [32.0]
x_g116: [37.0]
x_g117: [8.0]
x_g590: [39.0]
Objective: 11.395442669
